In [228]:
import glob
import json
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup
import re

from dotenv import load_dotenv
import os

from openai import OpenAI
from textwrap import dedent

from tqdm import tqdm

# 讀取 .env 檔案
load_dotenv()

True

In [229]:
working_options = {
    "crawler_GDPR": False,
}

# Spider all GDPR

In [230]:
def fetch_page_content(url):
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

def extract_links_from_table(soup, table_id):
    table = soup.find('table', {'id': table_id})
    if table:
        return table.find_all('a')
    return []

def extract_links_from_div(soup, div_class):
    div = soup.find('div', {'class': div_class})
    if div:
        return div.find_all('a')
    return []

def extract_text_from_spans(soup, span_class):
    spans = soup.find_all('span', {'class': span_class})
    return [span.text for span in spans][0]

def extract_list_items_from_ol(soup, div_class):
    div = soup.find('div', {'class': div_class})
    if div:
        ols = div.find_all('ol')
        return [li.text for ol in ols for li in ol.find_all('li')]
    return []

def extract_text_from_paragraphs(soup, div_class):
    div = soup.find('div', {'class': div_class})
    if div:
        paragraphs = div.find_all('p')
        return [p.text for p in paragraphs]
    return []

def prepare_data(links, recitals_links):
    data = []
    for link in links + recitals_links:
        text = link.text.strip()
        href = link['href']
        if "chapter" not in text.lower() and 'recitals' not in text.lower():
            current_temp = {'Number': text, 'Link': href}
            if 'art' in href:
                current_temp['Type'] = 'Article'
            elif 'recitals' in href:
                current_temp['Type'] = 'Recital'
            data.append(current_temp)
    return data

def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    return df

## Crawler URL and Article and Save

In [231]:
gdpr_url = "https://gdpr-info.eu/"
recitals_url = "https://gdpr-info.eu/recitals/"
    
def crawler_url():
    gdpr_soup = fetch_page_content(gdpr_url)
    recitals_soup = fetch_page_content(recitals_url)

    gdpr_links = extract_links_from_table(gdpr_soup, 'tablepress-12')
    recitals_links = extract_links_from_div(recitals_soup, 'widget-area recital-widget-area')

    print("gdpr count:", len(gdpr_links))
    print("recitals count:", len(recitals_links))

    data = prepare_data(gdpr_links, recitals_links)
    link_df = pd.DataFrame(data)

    print("GDPR Articles and Recitals Links saved to GDPR_Articles_Recitals_Links.xlsx")
    print(f"應有 99(GDPR) + 173(Recitals) = {99+173}, 實際有：{len(link_df)}")

    link_df.head()
    
    return link_df

def crawler_article(link_df):
    articles_and_recitals_list = []
    for index, row in link_df.iterrows():
        current_url = row['Link']
        current_gdpr_article_content = fetch_page_content(current_url)
        title = extract_text_from_spans(current_gdpr_article_content, span_class='dsgvo-title')
        content_item_list = extract_list_items_from_ol(current_gdpr_article_content, div_class='entry-content')
        content_paragraphs = extract_text_from_paragraphs(current_gdpr_article_content, div_class='entry-content')
        
        articles_and_recitals_list.append({
            'Number': row['Number'],
            'Link': row['Link'],
            'Type': row['Type'],
            'Title': title,
            'Content_Items': content_item_list if content_item_list else content_paragraphs
        })

    articles_df = pd.DataFrame(articles_and_recitals_list)
    return articles_df

    
if working_options["crawler_GDPR"]:
    
    # - Crawler GDPR Articles and Recitals Links
    link_df = crawler_url()
    
    # - Crawler GDPR Articles
    articles_df = crawler_article()
    
    # - Save to Excel
    with pd.ExcelWriter("GDPR_Articles_Recitals_Links.xlsx", engine='xlsxwriter') as writer:
        articles_df.to_excel(writer, index=False, sheet_name='Article & Recitals')


# Data Preprocessing

In [232]:
source_data_folder_name = 'batch0813_43'
files = glob.glob(f'{source_data_folder_name}/*/report_*.json')

# 提取重複使用的正則表達式
article_pattern = r'\b(Article|Articles|Art|art|article|articles|GDPR)\b'
recital_pattern = r'\b(recital|Recital|Rec|rec)\b'

# 定義一個函數來讀取和處理單個 JSON 文件
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # 處理 compliant 和 non_compliant 資料
    combined_df = pd.concat([
        create_dataframe(data.get('compliant', []), 'compliant'),
        create_dataframe(data.get('non_compliant', []), 'non-compliant')],
        ignore_index=True
    )
    
    # 增加一個欄位標示資料夾名稱
    folder_name = os.path.basename(os.path.dirname(file_path))
    combined_df['folder'] = folder_name
    
    return combined_df

# 定義一個函數來創建資料框並拆分條文
def create_dataframe(data_list, compliance_status):
    df = pd.DataFrame(data_list)
    
    if 'article numbers' not in df.columns:
        df['article numbers'] = ''
    
    split_df = split_articles(df)
    split_df['compliance_status'] = compliance_status
    
    return split_df

# 定義一個函數來拆分條文並新增 Type 欄位
# 定義一個函數來拆分條文並新增 Type 欄位
def split_articles(df):
    rows = []
    for _, row in df.iterrows():
        article_numbers = row['article numbers']
        
        # 確保 article_numbers 是一個可被處理的單一值或字符串
        if isinstance(article_numbers, (list, np.ndarray)):
            article_numbers = ', '.join(map(str, article_numbers))
        elif pd.isnull(article_numbers):
            article_numbers = ''
        else:
            article_numbers = str(article_numbers)
        
        articles = article_numbers.split(', ')
        for article in articles:
            new_row = row.copy()
            new_row['article numbers'] = article
            
            # 判斷是 Article 還是 Recital
            new_row['type'] = classify_article_type(article)
            
            rows.append(new_row)
    return pd.DataFrame(rows)


# 定義一個函數來分類 Article 或 Recital
def classify_article_type(article):
    if re.search(article_pattern, article, re.IGNORECASE):
        return 'Article'
    elif re.search(recital_pattern, article, re.IGNORECASE):
        return 'Recital'
    else:
        return 'Article' # 預設為 Article   

# 定義清理文章號碼的函數，只保留整數部分
def clean_article_number(article):
    match = re.match(r'^\d+', article)  # 只匹配整數部分
    return match.group(0) if match else ""

# 處理所有文件並合併結果
all_data = pd.DataFrame()
for file in files:
    combined_df = process_file(file)
    all_data = pd.concat([all_data, combined_df], ignore_index=True)

# 保留原始的 article numbers 並移除贅詞的版本
all_data['clean_article numbers'] = (
    all_data['article numbers']
    .str.replace(article_pattern, '', regex=True)  # 移除 Article 相關的贅詞
    .str.replace(recital_pattern, '', regex=True)  # 移除 Recital 相關的贅詞
    .str.replace(r'(?<!\d)\.(?!\d)', '', regex=True)  # 移除不在數字之間的點
    .str.strip()
)

# 將 clean_article numbers 轉換成 single_article
all_data['single_article'] = all_data['clean_article numbers'].apply(clean_article_number)
all_data = all_data[all_data['single_article'].notna() & (all_data['single_article'] != '')]
all_data['single_article'] = all_data['single_article'].astype(int)
all_data = all_data.sort_values(by='single_article')

# 打印數量並顯示頭部數據
print("所有數量:", len(all_data))
print("clean_article numbers:", len(all_data['article numbers'].unique()))

# 將合併後的 DataFrame 存成 Excel 文件
excel_file_path = f'./{source_data_folder_name}_cleaned_compliance_data_with_original.xlsx'
# all_data.to_excel(excel_file_path, index=False)
# all_data.head()


所有數量: 1143
clean_article numbers: 281


## Combine the GDPR truth article

In [233]:
gdpr_articles_recitals_links_df = pd.read_excel("./GDPR_Articles_Recitals_Links.xlsx")

for index, row in all_data.iterrows():
    # 找到相同的 single_article 和 type 的行
    matching_rows = gdpr_articles_recitals_links_df[
        (gdpr_articles_recitals_links_df['Number'] == row["single_article"]) &
        (gdpr_articles_recitals_links_df['Type'] == row["type"])
    ]
    
    # 如果有匹配的行，將 Content_Items 的值加入到 all_data 中
    if not matching_rows.empty:
        # 將匹配行的 Content_Items 內容拼接成一個字符串
        content_items = "; ".join(matching_rows['Content_Items'].astype(str))
        all_data.at[index, 'Content_Items'] = content_items
        
all_data.to_excel(excel_file_path, index=False)
all_data.head()

,section,article numbers,legal provisions,type,compliance_status,amend,folder,article_numbers,legal_provisions,clean_article numbers,single_article,Content_Items
111,Compliance with the law: Our privacy policy ha...,3,Territorial scope,Article,non-compliant,"The GDPR has extraterritorial applicability, m...",https___www_viatech_com_tw_company_overview_tw...,NaN,NaN,3,3,['This Regulation applies to the processing of...
1008,使用者可以經由瀏覽器的設定，取消或限制此項功能。,Article 4(11),The policy mentions that users can control coo...,Article,compliant,NaN,http___www_x_legend_tw_03financials_financials...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...
1138,Cookies會在不同時間到期，大多數cookies在瀏覽器關閉時到期，第三方cookies...,Article 4(11),"Conditions for consent, Definition of consent",Article,non-compliant,This section needs to be more specific about w...,https___www_panjit_com_tw_tw_Esg_stakeholder_c...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...
243,如您提供之資料包含第三人之個人資料時，您需確認該第三人已知悉本隱私權聲明內之權利，並擔保您已...,Articles 4(11),Data subject must be informed and provide cons...,Article,compliant,NaN,https___www_acergadget_com__chunks_merged,NaN,NaN,4(11),4,['‘personal data’ means any information relati...
684,"By accessing CyberTAN Services, you consent to...",Articles 4(11),"Consent: ensuring consent is freely given, spe...",Article,compliant,NaN,https___www_cybertan_com_tw_public_tw_about_06...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...


## Validation

In [234]:
# 取得 GPT-KEY
gpt_key = os.getenv('GPT_KEY')
if gpt_key is not None: print("Successfully set GPT_KEY")


def validate_prompt(original_section, original_article_number, actual_article=""):
    return dedent(f'''
    You are a professional legal practitioner.
    
    Reference=
    ```
    {actual_article}
    ```
    
    Section=
    ```
    {original_section}
    ```
    
    Use the reference to determine whether it complies with GDPR Article {original_article_number}. 
    
    ```
    OUTPUT=
    {{
    "is_correct": true/false
    }}
    ```
    
    The result returned is a single-line JSON format string, without line breaks or special characters.
    ''')

all_data['is_correct'] = None
prompt_list = []

for index, row in tqdm(all_data[:100].iterrows()):
    if row['compliance_status'] == 'non-compliant': 
        all_data.at[index, 'is_correct'] = str("non-compliant") 
        continue
    
    count = 0
    try:
        prompt_content = validate_prompt(row['section'], row['single_article'], row['Content_Items'])

        client = OpenAI(api_key=gpt_key)
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a GDPR checker."},
                {"role": "user", "content": prompt_content}
            ],
            temperature=0
        )
        
        for repeat_index in range(3):
            try:
                # 取得回覆
                response = completion.choices[0].message.content.strip()
                response_dict = json.loads(response)

                # 更新 DataFrame
                all_data.at[index, 'is_correct'] = False
                response_correct = str(response_dict.get('is_correct')) 
                if response_correct == 'True' : 
                    count += 1
                    if count > 1:
                        all_data.at[index, 'is_correct'] = str(response_dict.get('is_correct')) 
                        break
            except:
                continue
        
            prompt_list.append(prompt_content + "\n" + str(response_dict.get('is_correct')) )

    except Exception as e:
        all_data.at[index, 'is_correct'] = 'Error'

separator = '-' * 40
with open('output.txt', 'w', encoding='utf-8') as file:
    for prompt_content in prompt_list:
        file.write(prompt_content + '\n')
        file.write(separator + '\n')

# 保存 DataFrame 到 Excel
all_data.to_excel(excel_file_path, index=False)
all_data.head()

Successfully set GPT_KEY


100it [00:34,  2.87it/s]


,section,article numbers,legal provisions,type,compliance_status,amend,folder,article_numbers,legal_provisions,clean_article numbers,single_article,Content_Items,is_correct
111,Compliance with the law: Our privacy policy ha...,3,Territorial scope,Article,non-compliant,"The GDPR has extraterritorial applicability, m...",https___www_viatech_com_tw_company_overview_tw...,NaN,NaN,3,3,['This Regulation applies to the processing of...,non-compliant
1008,使用者可以經由瀏覽器的設定，取消或限制此項功能。,Article 4(11),The policy mentions that users can control coo...,Article,compliant,NaN,http___www_x_legend_tw_03financials_financials...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,True
1138,Cookies會在不同時間到期，大多數cookies在瀏覽器關閉時到期，第三方cookies...,Article 4(11),"Conditions for consent, Definition of consent",Article,non-compliant,This section needs to be more specific about w...,https___www_panjit_com_tw_tw_Esg_stakeholder_c...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,non-compliant
243,如您提供之資料包含第三人之個人資料時，您需確認該第三人已知悉本隱私權聲明內之權利，並擔保您已...,Articles 4(11),Data subject must be informed and provide cons...,Article,compliant,NaN,https___www_acergadget_com__chunks_merged,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,True
684,"By accessing CyberTAN Services, you consent to...",Articles 4(11),"Consent: ensuring consent is freely given, spe...",Article,compliant,NaN,https___www_cybertan_com_tw_public_tw_about_06...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,False


## To Json

In [235]:
filtered_data = all_data[~all_data['is_correct'].isin(['ERROR', 'NONE']) & (all_data['is_correct'] != False)]
print(filtered_data.head().to_dict())
print(f"總數：{len(filtered_data)}")
filtered_data.head()

{'section': {111: 'Compliance with the law: Our privacy policy has been compiled so as to comply with the law of every country or legal jurisdiction in which we aim to do business.', 1008: '使用者可以經由瀏覽器的設定，取消或限制此項功能。', 1138: 'Cookies會在不同時間到期，大多數cookies在瀏覽器關閉時到期，第三方cookies則由第三方網站控制。您可以透過調整瀏覽器的設定（例如Microsoft Internet Explorer、Firefox、Chrome）隨時停用這些cookies。在此提醒您停用cookies可能會影響正確使用本公司網站，若您停用了強制性cookies，您可能無法使用登入功能。', 243: '如您提供之資料包含第三人之個人資料時，您需確認該第三人已知悉本隱私權聲明內之權利，並擔保您已取得第三人之同意得授權我們依據本隱私權聲明內蒐集目的使用第三人之個人資料。', 361: '「個人資料」是指可以直接或間接地識別您個人身分的資料。'}, 'article numbers': {111: '3', 1008: 'Article 4(11)', 1138: 'Article 4(11)', 243: 'Articles 4(11)', 361: '4(1)'}, 'legal provisions': {111: 'Territorial scope', 1008: 'The policy mentions that users can control cookies through browser settings, aligning with the requirement for consent to cookie use.', 1138: 'Conditions for consent, Definition of consent', 243: 'Data subject must be informed and provide consent for data processing.', 361: 'Definition of p

,section,article numbers,legal provisions,type,compliance_status,amend,folder,article_numbers,legal_provisions,clean_article numbers,single_article,Content_Items,is_correct
111,Compliance with the law: Our privacy policy ha...,3,Territorial scope,Article,non-compliant,"The GDPR has extraterritorial applicability, m...",https___www_viatech_com_tw_company_overview_tw...,NaN,NaN,3,3,['This Regulation applies to the processing of...,non-compliant
1008,使用者可以經由瀏覽器的設定，取消或限制此項功能。,Article 4(11),The policy mentions that users can control coo...,Article,compliant,NaN,http___www_x_legend_tw_03financials_financials...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,True
1138,Cookies會在不同時間到期，大多數cookies在瀏覽器關閉時到期，第三方cookies...,Article 4(11),"Conditions for consent, Definition of consent",Article,non-compliant,This section needs to be more specific about w...,https___www_panjit_com_tw_tw_Esg_stakeholder_c...,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,non-compliant
243,如您提供之資料包含第三人之個人資料時，您需確認該第三人已知悉本隱私權聲明內之權利，並擔保您已...,Articles 4(11),Data subject must be informed and provide cons...,Article,compliant,NaN,https___www_acergadget_com__chunks_merged,NaN,NaN,4(11),4,['‘personal data’ means any information relati...,True
361,「個人資料」是指可以直接或間接地識別您個人身分的資料。,4(1),Definition of personal data,Article,compliant,NaN,https___www_flytech_com_tw_stakeholder_php_chu...,NaN,NaN,4(1),4,['‘personal data’ means any information relati...,True


In [236]:
import pandas as pd
import json

# 定義要輸出的 JSON 檔路徑
output_json_path = f"{source_data_folder_name}_combine_article_filter.json"

def combine_rows(group):
    return {
        "type": group.name[0],
        "single_article": group.name[1],
        "Content_Items": list(set(group['Content_Items'])),  # 使用 set 去重並轉換回列表
        "compliant": group[group['compliance_status'] == 'compliant'][['folder', 'section', 'legal provisions', 'amend']].to_dict(orient='records'),
        "non-compliant": group[group['compliance_status'] == 'non-compliant'][['folder', 'section', 'legal provisions', 'amend']].to_dict(orient='records')
    }

# 根據 type、single_article 和 Content_Items 分組並合併
combined_df = filtered_data.groupby(['type', 'single_article', 'Content_Items']).apply(combine_rows).reset_index(drop=True)

# 將結果轉換為 JSON 並輸出至檔案
combined_df.to_json(output_json_path, orient='records', force_ascii=False)

print(f"總數：{len(combined_df)}")
combined_df.head()

/var/folders/kc/jbnf20ls6hl2qsfkbcc8pb600000gn/T/ipykernel_53039/1439411299.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = filtered_data.groupby(['type', 'single_article', 'Content_Items']).apply(combine_rows).reset_index(drop=True)


總數：46


0    {'type': 'Article', 'single_article': 3, 'Cont...
1    {'type': 'Article', 'single_article': 4, 'Cont...
2    {'type': 'Article', 'single_article': 5, 'Cont...
3    {'type': 'Article', 'single_article': 6, 'Cont...
4    {'type': 'Article', 'single_article': 7, 'Cont...
dtype: object

# Clustering

In [237]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# import numpy as np
# import plotly.express as px   

In [238]:
# output_file_path = f'{source_data_folder_name}_cluster_compliance_data.xlsx' 
# cluster_writer_excel = pd.ExcelWriter(output_file_path, engine='xlsxwriter')

In [239]:
# # 讀取上傳的 Excel 文件
# file_path = excel_file_path
# compliance_data_df = pd.read_excel(file_path)

# # 將各欄位分別進行 TF-IDF 向量化
# def vectorize_column(data, column_name):
#     vectorizer = TfidfVectorizer(stop_words='english')
#     vectors = vectorizer.fit_transform(data[column_name].astype(str).tolist())
#     return vectors

# vector_section = vectorize_column(compliance_data_df, 'section')
# vector_article_numbers = vectorize_column(compliance_data_df, 'clean_article numbers')
# vector_legal_provisions = vectorize_column(compliance_data_df, 'legal provisions')

# # 合併各欄位的向量表示
# from scipy.sparse import hstack
# X_combined = hstack([vector_section, vector_article_numbers, vector_legal_provisions])

## K_means

手肘法（Elbow Method）：  
通過計算不同k值下的SSE（Sum of Squared Errors）來找到最佳k值，SSE會隨著k的增加而減小，當SSE減小幅度變緩時，即所謂的“手肘”點，即為最佳k值。

輪廓係數（Silhouette Coefficient）：  
輪廓係數能同時考慮簇內和簇間距離，其值在-1到1之間，越接近1說明聚類效果越好。可以計算不同k值下的平均輪廓係數來選擇最佳k值。

Calinski-Harabasz Index：  
該指標基於簇內和簇間距離計算，值越大越好。

In [240]:
# # 定義範圍
# max_k = 5  # 限制最大k值
# k_range = range(2, max_k)

# # 保存不同指標的結果
# sse = []
# silhouette_scores = []
# calinski_scores = []
# davies_scores = []

# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
#     clusters = kmeans.fit_predict(X_combined)
#     sse.append(kmeans.inertia_)
#     silhouette_scores.append(silhouette_score(X_combined, clusters))
#     calinski_scores.append(calinski_harabasz_score(X_combined.toarray(), clusters))
#     davies_scores.append(davies_bouldin_score(X_combined.toarray(), clusters))

# # 找到手肘點 (SSE)
# diff = np.diff(sse)
# diff_r = diff[1:] / diff[:-1]
# knee_point = np.argmin(diff_r) + 2

# # 找到最大輪廓係數點
# best_silhouette = np.argmax(silhouette_scores) + 2

# # 找到最大Calinski-Harabasz Index點
# best_calinski = np.argmax(calinski_scores) + 2

# # 找到最小Davies-Bouldin Index點
# best_davies = np.argmin(davies_scores) + 2

# # 綜合考慮這些指標，選擇最常出現的分群數量
# best_k_candidates = [knee_point, best_silhouette, best_calinski, best_davies]
# best_k = max(set(best_k_candidates), key=best_k_candidates.count)

# print(f"手肘法最佳分群數量: {knee_point}")
# print(f"最大輪廓係數最佳分群數量: {best_silhouette}")
# print(f"最大Calinski-Harabasz Index最佳分群數量: {best_calinski}")
# print(f"最小Davies-Bouldin Index最佳分群數量: {best_davies}")
# print(f"綜合考慮的最佳分群數量: {best_k}")

# # 使用K-means進行文本分群
# kmeans = KMeans(n_clusters=best_k, random_state=42, n_init='auto')  
# clusters = kmeans.fit_predict(X_combined)

# # 將分群結果添加回數據框
# compliance_data_df['cluster'] = clusters

# # 使用PCA進行降維
# pca = PCA(n_components=2)
# X_pca_2d = pca.fit_transform(X_combined.toarray())

# # 將降維結果添加回數據框
# compliance_data_df['pca-2d-one'] = X_pca_2d[:,0]
# compliance_data_df['pca-2d-two'] = X_pca_2d[:,1]

# # 使用Plotly進行互動式視覺化
# fig = px.scatter(
#     compliance_data_df, x='pca-2d-one', y='pca-2d-two', color='cluster',
#     hover_data=['section', 'clean_article numbers', 'legal provisions', 'compliance_status'],
#     title="PCA Clustering of Legal Provisions"
# )
# fig.show()


# compliance_data_df.to_excel(cluster_writer_excel, index=False, sheet_name='K_means')
# print(f"分群結果已保存到 {output_file_path}")


## AgglomerativeClustering

In [241]:
# from sklearn.cluster import AgglomerativeClustering

# # 定義範圍
# max_k = 5  # 限制最大k值
# k_range = range(2, max_k)

# # 保存不同指標的結果
# silhouette_scores = []
# calinski_scores = []
# davies_scores = []

# for k in k_range:
#     clustering = AgglomerativeClustering(n_clusters=k)
#     clusters = clustering.fit_predict(X_combined.toarray())
#     silhouette_scores.append(silhouette_score(X_combined, clusters))
#     calinski_scores.append(calinski_harabasz_score(X_combined.toarray(), clusters))
#     davies_scores.append(davies_bouldin_score(X_combined.toarray(), clusters))

# # 找到最大輪廓係數點
# best_silhouette = np.argmax(silhouette_scores) + 2

# # 找到最大Calinski-Harabasz Index點
# best_calinski = np.argmax(calinski_scores) + 2

# # 找到最小Davies-Bouldin Index點
# best_davies = np.argmin(davies_scores) + 2

# # 綜合考慮這些指標，選擇最常出現的分群數量
# best_k_candidates = [best_silhouette, best_calinski, best_davies]
# best_k = max(set(best_k_candidates), key=best_k_candidates.count)

# print(f"最大輪廓係數最佳分群數量: {best_silhouette}")
# print(f"最大Calinski-Harabasz Index最佳分群數量: {best_calinski}")
# print(f"最小Davies-Bouldin Index最佳分群數量: {best_davies}")
# print(f"綜合考慮的最佳分群數量: {best_k}")

# # 使用AgglomerativeClustering進行文本分群
# clustering = AgglomerativeClustering(n_clusters=best_k)  
# clusters = clustering.fit_predict(X_combined.toarray())

# # 將分群結果添加回數據框
# compliance_data_df['cluster'] = clusters

# # 使用PCA進行降維
# pca = PCA(n_components=2)
# X_pca_2d = pca.fit_transform(X_combined.toarray())

# # 將降維結果添加回數據框
# compliance_data_df['pca-2d-one'] = X_pca_2d[:,0]
# compliance_data_df['pca-2d-two'] = X_pca_2d[:,1]

# # 使用Plotly進行互動式視覺化
# fig = px.scatter(
#     compliance_data_df, x='pca-2d-one', y='pca-2d-two', color='cluster',
#     hover_data=['section', 'clean_article numbers', 'legal provisions', 'compliance_status'],
#     title="PCA Clustering of Legal Provisions"
# )
# fig.show()

# compliance_data_df.to_excel(cluster_writer_excel, index=False, sheet_name='Agglomerative')
# print(f"分群結果已保存到 {output_file_path}")

In [242]:
# cluster_writer_excel.close()